In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
!git clone https://github.com/n2ak/attentive_translator.git
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

'git' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import torch
vocab_size = 10
device = "cuda"
time = 10
data = torch.randint(0, vocab_size, (1, time)).long()
data.shape
with open("resources/input.txt") as f:
    data = f.read()
vocab = list(sorted(set(data)))
stoi = {v: i for (i, v) in enumerate(vocab)}
itos = {v: k for (k, v) in stoi.items()}
def encode(l): return [stoi[i] for i in l]
def decode(l): return ''.join([itos[i] for i in l])


context_length = 10
n_embeddings = 30
n_heads = 2
x_train = []
y_train = []

In [4]:
from src import GPTConfig,GPT
model = GPT(
    GPTConfig(
        N=1,
        vocab_size=len(vocab),
        n_embeddings=100,
        context_length=context_length,
        n_heads=10,
        ff_scale=4
    ),
    device,
).to(device)
optim = torch.optim.Adam(model.parameters())
(sum([p.numel() for p in model.parameters()]))

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
for i in range(len(data) - context_length):
    x = data[i: i+context_length]
    logits = data[i+1: i+1+context_length]
    x_train.append(encode(x))
    y_train.append(encode(logits))
x_train = torch.tensor(x_train, device=device)
y_train = torch.tensor(y_train, device=device)

In [ ]:
losses = []

In [ ]:
batch_size = 32
def get_batch(size):
    indices = torch.randint(len(data) - context_length, (size,))
    return (x_train[indices]), (y_train[indices])
import tqdm
model.train()
for _ in tqdm.tqdm(range(1000)):
    x, y = get_batch(batch_size)
    optim.zero_grad(set_to_none=True)
    logits, loss = model(x, y)
    loss.backward()
    optim.step()
    losses.append(loss.item())
import matplotlib.pyplot as plt
plt.plot(losses)
torch.mean(torch.tensor(losses[-100:]))

125265

In [ ]:
@torch.no_grad()
def generate(model, context, length):
    context = encode(context)
    print(context)
    prefix = 0
    if len(context) < context_length:
        prefix = context_length-len(context)
        context = ([stoi[" "]]*(prefix)) + context
    text = context
    model.eval()
    while (len(context)-prefix) < length:
        d = torch.tensor([text[-context_length:]], device=device)
        logits, _ = model(d)
        logits = logits[:, -1, :]
        next = torch.multinomial(logits.softmax(-1), 1)[0]
        text.append(next[0].item())
    text = text[prefix:]
    return decode(text)
generate(model, "ff",100)